In [23]:
from SeqEN2.utils.data_loader import read_json, write_json
import pandas as pd

In [24]:
import numpy as np

In [25]:
from IPython.display import clear_output

In [4]:
data = read_json("../data/KEGG_ACT_data_v5.json")

In [5]:
keys = list(data.keys())

In [6]:
for key in keys[:]:
    for item in [
        "URLS",
        "NAMES",
        "ORGANISM",
        "DEFINITION",
        "ORTHOLOGY",
        "PATHWAY",
        "MODULE",
        "NTSEQ",
    ]:
        if item in data[key].keys():
            del data[key][item]

In [7]:
len(keys)

94751

In [8]:
pfam_ACT_clan = [
    ("ACT", "PF01842", 66),
    ("ACT_3", "PF10000", 70),
    ("ACT_4", "PF13291", 80),
    ("ACT_5", "PF13710", 62),
    ("ACT_6", "PF13740", 76),
    ("ACT_7", "PF13840", 65),
    ("ACT_8", "PF19571", 141),
    ("NIL", "PF09383", 73),
    ("DUF493", "PF04359", 84),
    ("NikR_C", "PF08753", 77),
    ("ALS_ss_C", "PF10369", 74),
    ("Thr_dehydrat_C", "PF00585", 91),
]
pfam_acc = [item[1] for item in pfam_ACT_clan]
pfam_names = [item[0] for item in pfam_ACT_clan]
for key in keys[:]:
    data[key]["seq"] = data[key]["AASEQ"]["kegg"]
    del data[key]["AASEQ"]

    old_domains = []
    for domain in data[key]["MOTIF_old"]:
        if domain["family"] in pfam_names:
            old_domains.append(
                {"family": domain["family"], "pos": [domain["start"], domain["end"]]}
            )

    new_domains = []
    try:
        if isinstance(data[key]["MOTIF_new"], dict):
            for domain, value in data[key]["MOTIF_new"].items():
                if domain in pfam_acc:
                    family = value["id"]
                    if isinstance(value["locations"], list):
                        for loc in value["locations"]:
                            new_domains.append(
                                {"family": family, "pos": [loc["start"], loc["end"]]}
                            )
                    elif isinstance(value["locations"], dict):
                        new_domains.append(
                            {
                                "family": family,
                                "pos": [
                                    value["locations"]["start"],
                                    value["locations"]["end"],
                                ],
                            }
                        )
    except:
        print(data[key])

    data[key]["old_ACT"] = old_domains
    data[key]["new_ACT"] = new_domains
    del data[key]["MOTIF_old"]
    del data[key]["MOTIF_new"]

In [9]:
# for key in keys[:10]:
#     print(data[key])

In [11]:
new_data = {}
aa_keys = "WYFMILVAGPSTCEDQNHRK*"
w = 20
padding = w - 1
batch_num = 0
droped_seq = {}
remaining_seqs = len(data)
keys = np.random.permutation(list(data.keys()))

In [22]:
for key in keys[94116:]:
    item = data[key]
    seq = ("*" * padding) + item["seq"] + ("*" * padding)
    remaining_seqs -= 1
    try:
        seq_ndx = np.array([aa_keys.index(aa) for aa in seq])
    except ValueError:
        droped_seq[key] = {"item": item, "error": "non-standard residue"}
        continue

    ACT_p = np.zeros(len(seq))
    for domain in item["old_ACT"]:
        start, end = domain["pos"]
        start = padding + int(start)
        end = padding + int(end)
        ACT_p[start:end] += 1
    for domain in item["new_ACT"]:
        start, end = domain["pos"]
        start = padding + int(start)
        end = padding + int(end)
        ACT_p[start:end] += 1
    if np.max(ACT_p) == 0.0:
        droped_seq[key] = {"item": item, "error": "no ACT domain"}
        continue
    ACT_p = np.round(ACT_p / np.max(ACT_p), 2)

    n = 0
    while len(seq_ndx) >= w:
        p = np.round(np.mean(ACT_p[:w]), 2)
        one_p = np.round(1 - p, 2)
        features = list(seq_ndx[:w]) + [p, one_p]
        col_labels = [f"f{i}" for i in range(w)] + ["p", "n"]
        new_data[f"{key}_{n}"] = {
            col: value for col, value in zip(col_labels, features)
        }
        seq_ndx = seq_ndx[1:]
        ACT_p = ACT_p[1:]
        n += 1

    if len(new_data) > 1000000:
        df = pd.DataFrame.from_dict(new_data, orient="index")
        df.to_csv(f"../data/w_{w}_KeggSeq_ndx_wpACT/batch_{batch_num}.csv.gz")
        write_json(droped_seq, "../data/w_20_KeggSeq_ndx_wpACT/dropped_seqs.json")
        new_data = {}
        batch_num += 1
        print(remaining_seqs)
        clear_output(wait=True)

df = pd.DataFrame.from_dict(new_data, orient="index")
df.to_csv(f"../data/w_{w}_KeggSeq_ndx_wpACT/batch_{batch_num}.csv.gz")
write_json(droped_seq, "../data/w_20_KeggSeq_ndx_wpACT/dropped_seqs.json")
new_data = {}
print(remaining_seqs)
clear_output(wait=True)

0


In [102]:
w = 20
batch_num = 1
df = pd.concat(
    [
        pd.read_csv(f"../data/w_{w}_KeggSeq_ndx_wpACT/batch_{i}.csv.gz", index_col=0)
        for i in range(14, 20)
    ],
    ignore_index=True,
)

In [103]:
df

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f12,f13,f14,f15,f16,f17,f18,f19,p,n
0,20,20,20,20,20,20,20,20,20,20,...,20,20,20,20,20,20,20,3,0.0,1.0
1,20,20,20,20,20,20,20,20,20,20,...,20,20,20,20,20,20,3,7,0.0,1.0
2,20,20,20,20,20,20,20,20,20,20,...,20,20,20,20,20,3,7,5,0.0,1.0
3,20,20,20,20,20,20,20,20,20,20,...,20,20,20,20,3,7,5,6,0.0,1.0
4,20,20,20,20,20,20,20,20,20,20,...,20,20,20,3,7,5,6,6,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6001940,3,10,6,19,13,20,20,20,20,20,...,20,20,20,20,20,20,20,20,0.0,1.0
6001941,10,6,19,13,20,20,20,20,20,20,...,20,20,20,20,20,20,20,20,0.0,1.0
6001942,6,19,13,20,20,20,20,20,20,20,...,20,20,20,20,20,20,20,20,0.0,1.0
6001943,19,13,20,20,20,20,20,20,20,20,...,20,20,20,20,20,20,20,20,0.0,1.0


In [100]:
# df['pr'] = df.apply(lambda x: '_'.join(x.name.split('_')[:-1]) , axis=1) #
items = np.unique(["_".join(item.split("_")[:-1]) for item in df.index])
items

array(['100008853', '100011414', '100029571', ..., 'y1988', 'zj316_2287',
       'zj316_2325'], dtype='<U17')

In [91]:
# df.set_index('pr', append=True, inplace=True)
# df

In [68]:
new_df = df[df.index.str.contains("BLSMQ_2650")]

In [69]:
new_df

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f12,f13,f14,f15,f16,f17,f18,f19,p,n
BLSMQ_2650_0,20,20,20,20,20,20,20,20,20,20,...,20,20,20,20,20,20,20,3,0.0,1.0
BLSMQ_2650_1,20,20,20,20,20,20,20,20,20,20,...,20,20,20,20,20,20,3,16,0.0,1.0
BLSMQ_2650_2,20,20,20,20,20,20,20,20,20,20,...,20,20,20,20,20,3,16,16,0.0,1.0
BLSMQ_2650_3,20,20,20,20,20,20,20,20,20,20,...,20,20,20,20,3,16,16,18,0.0,1.0
BLSMQ_2650_4,20,20,20,20,20,20,20,20,20,20,...,20,20,20,3,16,16,18,17,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BLSMQ_2650_184,18,7,5,18,10,20,20,20,20,20,...,20,20,20,20,20,20,20,20,0.0,1.0
BLSMQ_2650_185,7,5,18,10,20,20,20,20,20,20,...,20,20,20,20,20,20,20,20,0.0,1.0
BLSMQ_2650_186,5,18,10,20,20,20,20,20,20,20,...,20,20,20,20,20,20,20,20,0.0,1.0
BLSMQ_2650_187,18,10,20,20,20,20,20,20,20,20,...,20,20,20,20,20,20,20,20,0.0,1.0


In [70]:
np.sum((new_df.iloc[0, :20] == new_df.iloc[1, :20]))

18

In [71]:
import torch

In [74]:
a = torch.tensor(new_df.iloc[0, :20].values)
b = torch.tensor(new_df.iloc[1, :20].values)

In [78]:
torch.sum(a == b).item()

18